# Import

In [9]:
import torch
from models import builer as builder
import argparse
from torchvision.transforms import transforms
import utils
import pandas as pd
import os
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

In [10]:
from PIL import Image
from tools import encode as en
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [5]:
cd imagenet-autoencoder

c:\Users\seung\Desktop\auto_image\Mixed-AutoEncoder\imagenet-autoencoder


# Prepare

In [38]:
X, _ = make_blobs(n_samples=300, centers=4, cluster_std=0.60, random_state=0)
args_dict = {
    "arch": "resnet34",
    "resume": "./caltech256-resnet34.pth",
    "parallel" : 0   
}
args_namespace = argparse.Namespace(**args_dict)

model = builder.BuildAutoEncoder(args_namespace)
def get_encode_vector(model, dataset):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    latent_vectors = []
    trans = transforms.Compose([
                transforms.Resize(256),                   
                transforms.CenterCrop(224),
                transforms.ToTensor()
                ])
    model.eval()
    for i in range(len(dataset)):
        path_img = dataset['path'].iloc[i]
        label = dataset['class'].iloc[i]
        img = Image.open(path_img).convert("RGB")
        img = trans(img).unsqueeze(0).to(device)
        latent_vector =  en.encode(model,img)
        latent_vectors.append(latent_vector.reshape(1,-1))
    
    return latent_vectors

In [6]:
data = pd.read_table('./list/weather_test_list.txt', names=["path", "class"], delimiter='\s')

C:\Users\seung\AppData\Local\Temp\ipykernel_24392\2406257182.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_table('./list/weather_test_list.txt', names=["path", "class"], delimiter='\s')


In [14]:
test_vector = get_encode_vector(model, data)
test_vector_concatenated = np.concatenate(test_vector, axis=0)
test_vector_concatenated.shape

(4133, 25088)

In [15]:
pca_resnet = PCA(n_components=700)
pca_resnet.fit(test_vector_concatenated)

test_transform_resnet = pca_resnet.transform(test_vector_concatenated)

In [17]:
kmeans_resnet34 = KMeans(n_clusters=7)
kmeans_resnet34.fit(test_transform_resnet)
y_kmeans = kmeans_resnet34.predict(test_transform_resnet)

In [19]:
len(y_kmeans)

4133

# Consensus clustering - evaluation

In [40]:
# y_kmeans : clustering output
# label : datas'class']
label = data['class']

In [37]:
def create_matrix(data, label):
    # the input is raw data. 
    if len(data.shape)!=1:
        raise ValueError("Input data must be a 1-dimensional array.")
    
    N = len(data)
    adj_matrix = np.zeros((N,N))
    ans_matrix = np.zeros((N,N))
    for row in range(N):
        for col in range(N):
            if data[row] == data[col]:
                adj_matrix[row,col] = 1
            if label[row]==label[col]:
                ans_matrix[row,col]=1
    return adj_matrix, ans_matrix

In [49]:
len(data)

4133

In [42]:
adj_matrix, ans_matrix = create_matrix(y_kmeans, label)

In [67]:
np.array_equal(ans_matrix,ans_matrix.T)

True

In [64]:
tt_matrix = (cp_matrix)/(adj_matrix + 1e-6)
tt_matrix = np.round(tt_matrix,decimals=0)
np.count_nonzero(tt_matrix==1)


717591

In [70]:
eval_matrix = ans_matrix /adj_matrix
eval_metrix = np.nan_to_num(eval_matrix, nan=0, posinf=0, neginf=0)
result_num = np.count_nonzero(eval_metrix==1)
print(f'estimated value : {(result_num-len(data))/(2*len(data))}')

C:\Users\seung\AppData\Local\Temp\ipykernel_24392\1360274982.py:1: RuntimeWarning: divide by zero encountered in true_divide
  eval_matrix = ans_matrix /adj_matrix
C:\Users\seung\AppData\Local\Temp\ipykernel_24392\1360274982.py:1: RuntimeWarning: invalid value encountered in true_divide
  eval_matrix = ans_matrix /adj_matrix


estimated value : 86.31236390031454


In [ ]:
def monti_evaluation(dataset, cluster_output):
    